SPDX-FileCopyrightText: 2021 NVIDIA CORPORATION

SPDX-License-Identifier: MIT

## Подготовка репозитория моделей


В качестве примера будет использовать модель [Hi-Fi GAN](https://ngc.nvidia.com/catalog/models/nvidia:nemo:tts_hifigan) из [NeMo](https://github.com/NVIDIA/NeMo).
Чтобы получить onnx этой модели нужно запустить контейнер NeMo
```
docker run --rm --gpus '"device=0"' -it --ipc=host \
-v $HOME/:/ext_home \
-v ${PWD}:${PWD} \
 -w ${PWD} \
--name $USER_nemo \
nvcr.io/nvidia/nemo:1.3.0
```

И в нём выполнить следующую ячейку

In [ ]:
from nemo.collections.tts.models import HifiGanModel

model = HifiGanModel.from_pretrained(model_name="tts_hifigan")
model.export("./hifigan.onnx")

model = HifiGanModel.from_pretrained(model_name="tts_hifigan")
model.export("./hifigan.pt")

После исполнения ячейки сверху в текущей директории появятся два файла `hifigan.onnx` и `hifigan.pt`

ONNX модель потребуется для экспериментов с Model Analyzer — этот инструмент помогает подобрать оптимальную конфигурацию для инференса в рамках одного бекенда. 
Для этого нужно скопировать `hifigan.onnx` в `model_repository/hifigan/1/model.onnx`

```
cp hifigan.onnx model_repository/hifigan/1/model.onnx
```

TorchScript модель понадобится позже, для экспериментов с Model Navigator, который может помочь подобрать наиболее оптимальный бекенд для модели.

## Model Analyzer Launch


Model Analyzer предназначен для выбора оптимальной конфигурации моделей для онлайн или оффлайн режима. Для этого он создаёт модели с различными конфигурациями, самостоятельно запускает контейнер с Triton'ом и использует Perf Analyzer для снятия метрик. Model Analyzer [Open Source](https://github.com/triton-inference-server/model_analyzer) и написан на Python.

Этот ноутбук рекомендуется исполнять в последнем контейнере с Triton SDK. При этом нужно подмонтировать полный путь до этого ноутбука под таким же путём, как и снаружи, чтобы model_analyzer мог удобно подмонтировать его к контейнеру с Triton'ом. Если запускать контейнер в папке с этим ноутбуком, рекомендуемая команда запуска

```
docker run --rm --gpus '"device=0"' -it --ipc=host --net=host \
-v /var/run/docker.sock:/var/run/docker.sock \
-v $(pwd):$(pwd) \
-v $HOME/:/ext_home \
--name $USER_triton_sdk \
nvcr.io/nvidia/tritonserver:21.08-py3-sdk
```

Для запуска ноутбука в контейнере необходимо дополнительно установить ipykernel
```
pip install ipykernel
```

Обратите внимание на подмонтированный docker.sock, который позволяет запускать контейнеры.

Если доступ на машину с GPU для инференса есть только через Kubernetes, это [тоже поддерживается](https://github.com/triton-inference-server/model_analyzer/blob/main/docs/kubernetes_deploy.md), но я не буду на этом останавливаться.

Перед запуском следующей команды следует убедиться, что на машине сейчас нет запущенного контейнера с name=tritonserver. Следующая команда остановит и удалит такой контейнер, если он был.
```
docker rm -f tritonserver 
```


## Model Analyzer Config


Из моделей, с которыми будет работать model analyzer, необходимо создать стандартный репозиторий моделей для Triton. У меня в репозитории в папке `model_repository` одна модель hifigan в двух версиях: 1 и 2. Версия 1 у меня напрямую экспортирована из NeMo, в версии 2 я вырезал бесполезный подграф. Сами файлы не прикладываю, чтобы не раздувать архив.

Есть два режима подбора параметров Model Analyzer: 
[автоматический](https://github.com/triton-inference-server/model_analyzer/blob/main/docs/config_search.md#automatic-configuration-search)
и [ручной](https://github.com/triton-inference-server/model_analyzer/blob/main/docs/config_search.md#Manual-Configuration-Search)
— [введение в команду model-analyzer profile](https://github.com/triton-inference-server/model_analyzer/blob/main/docs/cli.md#subcommand-profile)

Я подготовил для Hi-Fi GAN два конфига: ручной [profile_config_manual.yaml](profile_config_manual.yaml) и
автоматический [profile_config_auto.yaml](profile_config_auto.yaml). Они отличаются значением параметра `run_config_search_disable`. Больше подробностей о других параметрах [тут](https://github.com/triton-inference-server/model_analyzer/blob/main/docs/config.md#configuring-model-analyzer).

**В ручном конфиге**  Измеряются задержки и пропускная способность Hi-FI GAN на различных парметрах размера батча и одновременных запросов, и при различном количестве инстансов.

**В автоматическом конфиге** эти же параметры, а так же наличие динамического батчинга, подбираются в автоматическом режиме.

**В обоих конфигах необходимо задать полный путь до выходного репозитория моделей**, в котором можно будет посмотреть измеряемые конфиги моделей. 

Обратите внимание на задание шейпов в конфиге.

Следующая команда запускает Model Analyzer с ручным конфигом. Если захочется приостановить выполнение команды, можно послать ей сигнал `SIGINT`. Это можно сделать либо нажав `Ctrl+C`, либо нажав на знак «стоп» рядом с ячейкой, либо (самое надёжное), выполнить в терминале внутри контейнера

```
kill -INT $(ps aux | grep model-ana | grep python | sed "s/^[[:alnum:]]*[[:space:]]*\([[:digit:]]*\).*/\1/")
```

При этом в логе должно появиться
```
INFO[analyzer_state_manager.py:174] Received SIGINT 1/3. Will attempt to exit after current measurement.
```

Это означает, что model-analyzer дождётся окончания текущего измерения и после этого сохранит текущий статус в checkpoint, из которого можно уже сделать предварительный анализ (см. дальше)

In [ ]:
!model-analyzer profile -f profile_config_manual.yaml

**После завершения профилировки** необходимо запустить анализ. Он выполнится очень быстро, никакой нагрузки на GPU он не создаёт.
Самые интересные результаты окажутся в папке results (сейчас там лежат пример результатов на NVIDIA A10). PDF-отчёт будет лежать в папке reports.

In [ ]:
!model-analyzer analyze -f profile_config_manual.yaml

In [ ]:
# чтобы верифицировать yaml синтаксически
import yaml
with open("profile_config_manual.yaml") as pc:
    pc = yaml.safe_load(pc)

## Model Navigator

Model Navigator не установлен в контейнере с Triton SDK. Его контейнер необходимо собрать самостоятельно из исходников. [Инструкция.](https://github.com/triton-inference-server/model_navigator/blob/main/docs/quick_start.md#install-the-triton-model-navigator-and-run-container)

```bash
git clone https://github.com/triton-inference-server/model_navigator.git
# Optional
# git checkout v0.2.2
make docker
cd ..

docker run -it --rm \
 --gpus 1 \
 -v /var/run/docker.sock:/var/run/docker.sock \
 -v ${PWD}:${PWD} \
 -w ${PWD} \
 --net host \
 --name model-navigator \
 model-navigator /bin/bash
```

Конфигурация для Model Navigator хранится в файле [navigator_config.yaml](navigator_config.yaml). При работе Model Navigator склонен перезаписывать этот файл, поэтому команда запуска внутри контейнера предлагается такая:

```
cp navigator_config.yaml navigator_config_run.yaml; model-navigator run --config-path navigator_config_run.yaml
```